In [1]:
# Imports
from tkinter import filedialog, Tk
import yaml
from os import path, listdir, system
from shutil import copytree, rmtree, copy
from vsrc.constants import DATA
import subprocess
%load_ext autoreload
%gui tk

In [2]:
# Helper Functions
def save_no_overwrite(file: str) -> str:
    suffix = 0
    name, extension = path.splitext(file)
    while path.exists(file):
        file = f"{name}_{suffix}{extension}"
        suffix += 1
    return file

def open_file():
    root = Tk()
    filename = filedialog.askopenfilename(title='Select Model Or Checkpoint', initialdir='./models/', filetypes=[('Model Files', '*.pt')])
    root.destroy()
    return filename


In [2]:
# Constants
CHECKPOINT_PATH, RECIPE_PATH, LOGS_PATH = './models/checkpoints/', './models/recipes/', './models/logs/'
IMAGE_SIZE = 250

# Derived Constants

In [7]:
TRAIN_PARAM = {
    # Model definition
    'model': 'zoo:yolov8-m-coco-pruned80_quantized',
    'recipe': 'zoo:yolov8-m-voc_coco-pruned80_quantized',
    'data': 'coco128.yaml',
    'resume': False,
    # Paths
    'project' : LOGS_PATH,
    'name': 'demo_test',
    # Training Parameters
    'batch_size': 32,
    'imgsz': IMAGE_SIZE,
    'epochs': 16,
    'cos_lr': False,
    # Misc
    'workers': 8,
    'device': '0',
    'verbose': True,
}

command_str = f"sparseml.ultralytics.train --model {TRAIN_PARAM['model']} --recipe {TRAIN_PARAM['recipe']} --data {TRAIN_PARAM['data']} {'--resume' if TRAIN_PARAM['resume'] else ''} --project {TRAIN_PARAM['project']} --name {TRAIN_PARAM['name']} --batch {TRAIN_PARAM['batch_size']} --imgsz {TRAIN_PARAM['imgsz']} --epochs {TRAIN_PARAM['epochs']} {'--cos' if TRAIN_PARAM['cos_lr'] else ''} --workers {TRAIN_PARAM['workers']} --device {TRAIN_PARAM['device']} {'--verbose' if TRAIN_PARAM['verbose'] else ''}"
print(command_str)

sparseml.ultralytics.train --model zoo:yolov8-m-coco-pruned80_quantized --recipe zoo:yolov8-m-voc_coco-pruned80_quantized --data coco128.yaml  --project ./models/logs/ --name demo_test --batch 32 --imgsz 250 --epochs 16  --workers 8 --device 0 --verbose


In [8]:
process = subprocess.Popen(command_str, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)

# Stream the output
for line in process.stdout:
    print(line, end='')

# Close the process
process.stdout.close()
process.wait()

### Identifier

In [28]:
# Identifier Recipe
DATASET_FOLDER = './dataset/all'
YAML_RECIPE = {
    'path' : DATASET_FOLDER,
    'train' : 'train',
    'val' : 'test',
    'names' : [{ DATA[name]["num_label"] : name} for name in DATA],
}
# Copy all component data to all folder
if path.exists(DATASET_FOLDER): rmtree(DATASET_FOLDER)
# Get all component images from all folders
images, labels = [], []
for folder in listdir('./dataset'):
    if folder in ['template', 'test.png']: continue
    # Append all files in folder to images
    imgfiles = listdir(path.join('./dataset', folder, 'imgs'))
    labfiles = listdir(path.join('./dataset', folder, 'labels'))
    for file in imgfiles:
        images.append(path.join('./dataset', folder, 'imgs', file))
    for label in labfiles:
        labels.append(path.join('./dataset', folder, 'labels', label))
    print(f"Found {len(imgfiles)} images and {len(labfiles)} labels in {folder}")
# Copy all fules to new folder
copytree('./dataset/template', DATASET_FOLDER)
# Copy all images to new folder
for image in images:
    copy(image, path.join(DATASET_FOLDER, 'imgs'))
# Copy all labels to new folder
for label in labels:
    copy(label, path.join(DATASET_FOLDER, 'labels'))

Found 2 images and 1 labels in resistor


### Load Models

In [14]:
# Base Model
MODEL_PATH, MODEL_NAME = path.split(open_file())
# YAML Saving
YAML_FILEPATH = save_no_overwrite(path.join(RECIPE_PATH, f'{MODEL_NAME}.yaml'))
with open(YAML_FILEPATH, 'w') as f:
    yaml.dump(YAML_RECIPE, f)